In [1]:
!pip install beir --quiet
!pip install datasets --quiet

     |████████████████████████████████| 52 kB 700 kB/s 
     |████████████████████████████████| 78 kB 6.7 MB/s 
     |████████████████████████████████| 8.4 MB 42.7 MB/s 
     |████████████████████████████████| 379 kB 46.1 MB/s 
     |████████████████████████████████| 4.9 MB 37.9 MB/s 
     |████████████████████████████████| 3.1 MB 19.6 MB/s 
     |████████████████████████████████| 3.3 MB 40.8 MB/s 
     |████████████████████████████████| 1.2 MB 35.8 MB/s 
     |████████████████████████████████| 59 kB 6.1 MB/s 
     |████████████████████████████████| 596 kB 45.9 MB/s 
     |████████████████████████████████| 895 kB 33.1 MB/s 
     |████████████████████████████████| 290 kB 5.3 MB/s 
     |████████████████████████████████| 243 kB 44.7 MB/s 
     |████████████████████████████████| 1.1 MB 30.6 MB/s 
     |████████████████████████████████| 132 kB 50.2 MB/s 
     |████████████████████████████████| 192 kB 47.2 MB/s 
     |████████████████████████████████| 160 kB 46.6 MB/s 
     |███████████████

In [2]:
from datasets import load_dataset, load_metric
from beir import util
from beir.datasets.data_loader import GenericDataLoader
from sentence_transformers import SentenceTransformer
import sentence_transformers
import torch
import numpy as np


import random
import pandas as pd

# Fonctions utiles

In [3]:
import sys

def progressbar(it, prefix="", size=60, file=sys.stdout):
    count = len(it)
    def show(j):
        x = int(size*j/count)
        file.write("%s[%s%s] %i/%i\r" % (prefix, "#"*x, "."*(size-x), j, count))
        file.flush()        
    show(0)
    for i, item in enumerate(it):
        yield item
        show(i+1)
    file.write("\n")
    file.flush()

# Chargement des données

On récupère le jeu de validation de `squad_v2` ainsi que le dataset `DBpedia`

In [4]:
datasets = load_dataset("squad_v2", split='validation')

Downloading:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/9.55M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/801k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


In [5]:
datasets = pd.DataFrame(datasets)

In [6]:
datasets

,id,title,context,question,answers
0,56ddde6b9a695914005b9628,Normans,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,"{'text': ['France', 'France', 'France', 'Franc..."
1,56ddde6b9a695914005b9629,Normans,The Normans (Norman: Nourmands; French: Norman...,When were the Normans in Normandy?,"{'text': ['10th and 11th centuries', 'in the 1..."
2,56ddde6b9a695914005b962a,Normans,The Normans (Norman: Nourmands; French: Norman...,From which countries did the Norse originate?,"{'text': ['Denmark, Iceland and Norway', 'Denm..."
3,56ddde6b9a695914005b962b,Normans,The Normans (Norman: Nourmands; French: Norman...,Who was the Norse leader?,"{'text': ['Rollo', 'Rollo', 'Rollo', 'Rollo'],..."
4,56ddde6b9a695914005b962c,Normans,The Normans (Norman: Nourmands; French: Norman...,What century did the Normans first gain their ...,"{'text': ['10th century', 'the first half of t..."
...,...,...,...,...,...
11868,5737aafd1c456719005744ff,Force,"The pound-force has a metric counterpart, less...",What is the seldom used force unit equal to on...,"{'text': ['sthène', 'sthène', 'sthène', 'sthèn..."
11869,5ad28ad0d7d075001a4299cc,Force,"The pound-force has a metric counterpart, less...",What does not have a metric counterpart?,"{'text': [], 'answer_start': []}"
11870,5ad28ad0d7d075001a4299cd,Force,"The pound-force has a metric counterpart, less...",What is the force exerted by standard gravity ...,"{'text': [], 'answer_start': []}"
11871,5ad28ad0d7d075001a4299ce,Force,"The pound-force has a metric counterpart, less...",What force leads to a commonly used unit of mass?,"{'text': [], 'answer_start': []}"


In [7]:
dataset = "dbpedia-entity"
url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset)
data_path = util.download_and_unzip(url, "datasets")
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")

datasets/dbpedia-entity.zip:   0%|          | 0.00/610M [00:00<?, ?iB/s]

  0%|          | 0/4635922 [00:00<?, ?it/s]

# Preprocessing
A partir du dataset `DPpedia` on choisi 10000 topics aléatoires de plus de 50 mots qu'on va ajouter au dataset `SquadV2`. Les contextes de `SQuAD` sont labélisés avec un 1 et ceux de `DBpedia` avec un 0.

In [8]:
corpus_keys = list(corpus.keys())
contexts = list(zip(datasets['context'].tolist(), np.ones(len(datasets), dtype=int)))
for i in range(10000):
  choice = random.choice(corpus_keys)
  while len(corpus[choice]['text'].split()) < 50:
    choice = random.choice(corpus_keys)
  contexts.append((corpus[choice]['text'], 0))
random.shuffle(contexts)

In [9]:
contexts[:5]

[('Marcellus Graeme Boss (January 21, 1901 - March 21, 1967) was an interim Governor of Guam from November 14, 1959 until his resignation on August 22, 1960. Boss first entered Guamanian administration with a 1957 appointment by President Dwight D. Eisenhower to the post of Secretary of Guam. He also served as a Republican member of the Kansas State Senate prior to becoming involved in the government of Guam.',
  0),
 ('Many faults are able to produce a magnitude 6.7+ earthquake, such as the San Andreas Fault, which can produce a magnitude 8.0 event. Other faults include the San Jacinto Fault, the Puente Hills Fault, and the Elsinore Fault Zone. The USGS has released a California Earthquake forecast which models Earthquake occurrence in California.',
  1),
 ('Gerben Karstens (born 14 January 1942) is a former professional racing cyclist from the Netherlands, who won the gold medal in the 100 km team trial at the 1964 Summer Olympics, alongside Bart Zoet, Evert Dolman, and Jan Pieterse.

Au final on se retrouve avec +- 10000 contextes différents

In [10]:
len(contexts)

21873

# Projection 

Une fois notre corpus complet on le projette dans un espace vectoriel en utilisant un Ms MARCO model.

In [11]:
model = SentenceTransformer('msmarco-roberta-base-v3')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.70k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/678 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
contexts_embeddings = model.encode(list(zip(*contexts))[0], convert_to_numpy=True)

In [14]:
! pip install transformers --quiet
! pip install sentence-transformers --quiet
! pip install annoy --quiet
!pip install datasets --quiet

     |████████████████████████████████| 646 kB 5.3 MB/s 


In [22]:
from sentence_transformers import SentenceTransformer, util
import os
import csv
import pickle
import time
import torch
from annoy import AnnoyIndex

In [23]:
model_name = 'msmarco-roberta-base-v3'
max_corpus_size = 100000
n_trees = 256 
embedding_size = 768    #Size of embeddings
top_k_hits = 10         #Output k hits

annoy_index_path = 'quora-embeddings-{}-size-{}-annoy_index-trees-{}.ann'.format(model_name.replace('/', '_'), max_corpus_size,n_trees)

embedding_cache_path = 'quora-embeddings-{}-size-{}.pkl'.format(model_name.replace('/', '_'), max_corpus_size)

In [24]:
# Get all unique sentences from the file
corpus_sentences = list(zip(*contexts))[0]
corpus_sentences = list(corpus_sentences)

In [25]:
print("Store file on disc")
with open(embedding_cache_path, "wb") as fOut:
  pickle.dump({'sentences': corpus_sentences, 'embeddings': contexts_embeddings}, fOut)

Store file on disc


In [26]:
# Create Annoy Index
print("Create Annoy index with {} trees. This can take some time.".format(n_trees))
annoy_index = AnnoyIndex(embedding_size, 'angular')

for i in range(len(contexts_embeddings)):
    annoy_index.add_item(i, contexts_embeddings[i])

annoy_index.build(n_trees)
annoy_index.save(annoy_index_path)

contexts_embeddings = torch.from_numpy(contexts_embeddings)

Create Annoy index with 256 trees. This can take some time.


In [27]:
def nearest_neighbour(inp_question, display = False):

  recip_rank = 0.00000

  start_time = time.time()
  question_embedding = model.encode(inp_question)

  corpus_ids, scores = annoy_index.get_nns_by_vector(question_embedding, top_k_hits, include_distances=True)
  hits = []
  for id, score in zip(corpus_ids, scores):
    hits.append({'corpus_id': id, 'score': 1-((score**2) / 2)})

  end_time = time.time()

  for i in range(top_k_hits):
    if hits[i]['score'] == 1:
      recip_rank = (top_k_hits - i) / float(top_k_hits)
      break

  if display:
    print("Input question:", inp_question)
    print("Results (after {:.3f} seconds):".format(end_time-start_time))
    for hit in hits[0:top_k_hits]:
      print("\t{:.3f}\t{}".format(hit['score'], corpus_sentences[hit['corpus_id']]))

  
  # Approximate Nearest Neighbor (ANN) is not exact, it might miss entries with high cosine similarity
  # Here, we compute the recall of ANN compared to the exact results
  correct_hits = util.semantic_search(question_embedding, contexts_embeddings, top_k=top_k_hits)[0]
  correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])

  #Compute recall
  ann_corpus_ids = set(corpus_ids)
  if display:
    if len(ann_corpus_ids) != len(correct_hits_ids):
      print("Approximate Nearest Neighbor returned a different number of results than expected")

  recall = len(ann_corpus_ids.intersection(correct_hits_ids)) / len(correct_hits_ids)
  if display :
    print("\nApproximate Nearest Neighbor Recall@{}: {:.2f}".format(top_k_hits, recall * 100))

  if recall < 1:
    if display:
      print("Missing results:")
    for hit in correct_hits[0:top_k_hits]:
      if hit['corpus_id'] not in ann_corpus_ids and display:
        print("\t{:.3f}\t{}".format(hit['score'], corpus_sentences[hit['corpus_id']]))
  
  if display:
    print("\n\n========\n")
  return recip_rank

In [28]:
def compute_mrr(dataset):
  recip_ranks = 0.00000
  for row in dataset:
    recip_ranks += nearest_neighbour(row[0])

  return recip_ranks / float(len(dataset))

In [29]:
nearest_neighbour("The Architect of the City of Paris (Architecte de la ville de Paris) is the incumbent of a municipal position, responsible for the design and construction of civic projects in Paris, France.In the Ancien Régime in France, the position of Bâtiments du Roi oversaw the construction and maintenance of the King's properties in and around Paris.  This position lasted from 1620 through 1789.", True)

Input question: The Architect of the City of Paris (Architecte de la ville de Paris) is the incumbent of a municipal position, responsible for the design and construction of civic projects in Paris, France.In the Ancien Régime in France, the position of Bâtiments du Roi oversaw the construction and maintenance of the King's properties in and around Paris.  This position lasted from 1620 through 1789.
Results (after 0.053 seconds):
	0.556	Elies Rogent i Amat (Barcelona 18 July 1821 – Barcelona 21 February 1897), was a Catalan architect of Spanish nationality. He studied at the school of Architecture in Madrid, from which he graduated on 20 February 1851. He was appointed as the director of the Barcelona Provincial School of Architecture in 1871.An admirer of Eugène Viollet-le-Duc, he passed on his passion for mediaeval architecture through his teaching work.
	0.423	NAC Architecture is a design firm with over 140 architects, engineers, interior designers and support staff, with offices i

0.0